In [0]:
df_silver = spark.read.format('parquet')\
                  .option('inferSchema',True)\
                  .load('abfss://silver@saprojectyt.dfs.core.windows.net/carsales/sales.parquet')

df_silver.display()

In [0]:
df_dealer = spark.sql('SELECT * FROM youtube_project.gold.dim_dealer')
df_model = spark.sql('SELECT * FROM youtube_project.gold.dim_model')
df_branch = spark.sql('SELECT * FROM youtube_project.gold.dim_branch')
df_date = spark.sql('SELECT * FROM youtube_project.gold.dim_date')

In [0]:
df_fact = df_silver.join(df_branch,df_branch['Branch_ID'] == df_silver['Branch_ID'],how='left')\
                   .join(df_dealer,df_dealer['Dealer_ID'] == df_silver['Dealer_ID'],how='left')\
                   .join(df_model,df_model['Model_ID'] == df_silver['Model_ID'],how='left')\
                   .join(df_date,df_date['Date_ID'] == df_silver['Date_ID'],how='left')\
                   .select(df_silver['Revenue'],df_silver['Units_Sold'],df_silver['Revenue_perUnit'],df_branch['dim_branch_key'],df_dealer['dim_dealer_key'],df_model['dim_model_key'],df_date['dim_date_key']).where(df_date['dim_date_key'].isNotNull())

In [0]:
df_fact.display()

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists('youtube_project.gold.factSales'):
    deltaTable = DeltaTable.forName(spark,'youtube_project.gold.factSales')

    deltaTable.alias('trg').merge(df_fact.alias('src'),'trg.dim_branch_key = src.dim_branch_key AND trg.dim_dealer_key = src.dim_dealer_key AND trg.dim_model_key = src.dim_model_key AND trg.dim_date_key = src.dim_date_key')\
                .whenMatchedUpdateAll()\
                .whenNotMatchedInsertAll()\
                .execute()

else:
    df_fact.write.format('delta')\
                .mode('Overwrite')\
                .option('path','abfss://gold@saprojectyt.dfs.core.windows.net/factSales')\
                .saveAsTable('youtube_project.gold.factSales')    

In [0]:
%sql
SELECT * FROM youtube_project.gold.factsales